In [1]:
import rasterio
import glob
import os,sys
from matplotlib import pyplot as plt
import numpy as np
import fiona
from shapely.geometry import shape
import shapely
from rasterio.mask import mask
from pyproj import Proj, transform

from torch import nn
from torch.nn import functional as F
import torch
from torchvision import models
import torchvision
from torchvision.transforms import ToTensor, Normalize, Compose
from torch.utils.data import DataLoader

from unet_models import unet11, unet11_MS

from test_unet_helpers import calcXYfromRC, checkWindow, gtDatasetSampler2, DigitalGlobeSamplerTensor, load_pretrained_weights

from utils import variable
from scipy import misc

from gbdxtools import Interface, CatalogImage

from collections import OrderedDict

from datetime import datetime

In [2]:
# specify scene id for DG data
dg_scene_id = '1030010057062200' # 2016
# dg_scene_id = '104001000AA4B500' # 2015
# dg_scene_id = '103001001A425F00' #napoli 2011

# # specify image for 2.0 meter analysis
# gt_image_2 = '../rasters/union_impervious_raster_2_0_0_wgs84.tif' # desktop

# # specify images for 0.5 meter analysis
# gt_image_05 = '../rasters/union_impervious_raster_0_5.tif' # desktop

# # specify the shapefile
# shpfile = '../union/union.shp' # desktop

# with fiona.open(shpfile) as shp:
#     crs = shp.crs
#     shp_bounds = shp.bounds

In [3]:
# connect to gbdx
gbdx = Interface()

In [6]:
gbdx.ordering.order(dg_scene_id)

'e209b5dd-27e1-4af3-89ef-7bb8c92a3058'

In [9]:
orderid = 'e209b5dd-27e1-4af3-89ef-7bb8c92a3058'
while gbdx.ordering.status(orderid)[0]['state'] != 'delivered':
    if datetime.now().second % 30 == 0:
        print(gbdx.ordering.status(orderid))
        
print('finished', gbdx.ordering.status(orderid))

[{'acquisition_id': '1030010057062200', 'state': 'submitted', 'location': 'not_delivered'}]
[{'acquisition_id': '1030010057062200', 'state': 'submitted', 'location': 'not_delivered'}]
[{'acquisition_id': '1030010057062200', 'state': 'submitted', 'location': 'not_delivered'}]
[{'acquisition_id': '1030010057062200', 'state': 'submitted', 'location': 'not_delivered'}]
[{'acquisition_id': '1030010057062200', 'state': 'submitted', 'location': 'not_delivered'}]
[{'acquisition_id': '1030010057062200', 'state': 'submitted', 'location': 'not_delivered'}]
[{'acquisition_id': '1030010057062200', 'state': 'submitted', 'location': 'not_delivered'}]
[{'acquisition_id': '1030010057062200', 'state': 'submitted', 'location': 'not_delivered'}]
[{'acquisition_id': '1030010057062200', 'state': 'submitted', 'location': 'not_delivered'}]
[{'acquisition_id': '1030010057062200', 'state': 'submitted', 'location': 'not_delivered'}]
[{'acquisition_id': '1030010057062200', 'state': 'submitted', 'location': 'not_d

In [23]:
# get the dask array for the 8 band MS image
img_2m = CatalogImage(dg_scene_id, band_type='MS', acomp=True)
rows, cols = img_2m.shape[1:]
chips = img_2m.window_cover((256,256), pad=False)
num_chips = sum(1 for i in chips)
print(num_chips) # 96473

#np.mean(img_2m, axis=(1,2)).compute(), np.std(img_2m, axis=(1,2)).compute()

5985


In [24]:
from torchsat.models.segmentation.unet import UNetResNet

model_path = '../IS_segmentation/model_improvements/jstars_revisions/files_MS_aug_bgrn1/jstars_bgrn1_ms_aug_ep500_step8000_b32.pt'

model = UNetResNet(encoder_depth=152, num_classes=1, in_channels=4, num_filters=32, dropout_2d=0.2,
                 pretrained=False, is_deconv=True)

state_dict = torch.load(str(model_path), map_location=torch.device('cpu')) # added map_location for new pytorch!
new_state_dict = OrderedDict()
for k, v in state_dict['model'].items():
    name = k[7:] # remove 'module.' of dataparallel
    new_state_dict[name]=v

model.load_state_dict(new_state_dict)
model.eval()


UNetResNet(
  (encoder): ResNet(
    (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
       

False

# Load the model and the model state from the .pt file. It may need to be downloaded from AWS S3

In [25]:
## this cell uses GBDX window_cover method

## write out with gdal
import gdal, osr

def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):

    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

chips = img_2m.window_cover((256,256), pad=False)
# iterate over the image windows, getting the 4 bands
for i,chip in enumerate(chips):

    rname = '../tiles_256x256/denver_2015_t{0:0>6}_{1}.tif'.format(i+1, dg_scene_id)
    
    if not os.path.exists(os.path.dirname(rname)):
        print('creating tile directory')
        os.makedirs(os.path.dirname(rname))
    
    if os.path.exists(rname):
        continue

    print('on {} of {}'.format(i+1, num_chips))
    img_arr = (chip[[1,2,4,6], :, :].compute()/10000).astype('float32') #B-G-R-N1
    #img_arr = img_transform(torch.from_numpy(img_arr))

    #input_img = torch.unsqueeze(variable(img_arr, volatile=True), dim=0)
    input_img = torch.unsqueeze(variable(torch.from_numpy(img_arr), volatile=True), dim=0).cpu()
    del img_arr

    # model inference
    big_mask = model(input_img)

    # file creation
    raster_id = os.path.basename(model_path).split('.')[0]
    rname = '../tiles_256x256/denver_2016_t{0:0>6}_{1}.tif'.format(i+1, dg_scene_id)
    aff = chip.affine
    raster_origin = (aff.c, aff.f)
    pixel_height = aff.e
    pixel_width = aff.a
    array2raster(rname, raster_origin, pixel_width, pixel_height, big_mask.cpu().detach().numpy().squeeze())

    del input_img
    del big_mask


    torch.cuda.empty_cache()

creating tile directory
on 1 of 5985


/home/ubuntu/segmentation/IS_UNet_Resnet152/utils.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return cuda(Variable(x, volatile=volatile))


on 2 of 5985
on 3 of 5985
on 4 of 5985
on 5 of 5985
on 6 of 5985
on 7 of 5985
on 8 of 5985
on 9 of 5985
on 10 of 5985
on 11 of 5985
on 12 of 5985
on 13 of 5985
on 14 of 5985
on 15 of 5985
on 16 of 5985
on 17 of 5985
on 18 of 5985
on 19 of 5985
on 20 of 5985
on 21 of 5985
on 22 of 5985
on 23 of 5985
on 24 of 5985
on 25 of 5985
on 26 of 5985
on 27 of 5985
on 28 of 5985
on 29 of 5985
on 30 of 5985
on 31 of 5985
on 32 of 5985
on 33 of 5985
on 34 of 5985
on 35 of 5985
on 36 of 5985
on 37 of 5985
on 38 of 5985
on 39 of 5985
on 40 of 5985
on 41 of 5985
on 42 of 5985
on 43 of 5985
on 44 of 5985
on 45 of 5985
on 46 of 5985
on 47 of 5985
on 48 of 5985
on 49 of 5985
on 50 of 5985
on 51 of 5985
on 52 of 5985
on 53 of 5985
on 54 of 5985
on 55 of 5985
on 56 of 5985
on 57 of 5985
on 58 of 5985
on 59 of 5985
on 60 of 5985
on 61 of 5985
on 62 of 5985
on 63 of 5985
on 64 of 5985
on 65 of 5985
on 66 of 5985
on 67 of 5985
on 68 of 5985
on 69 of 5985
on 70 of 5985
on 71 of 5985
on 72 of 5985
on 73 of 5985


on 556 of 5985
on 557 of 5985
on 558 of 5985
on 559 of 5985
on 560 of 5985
on 561 of 5985
on 562 of 5985
on 563 of 5985
on 564 of 5985
on 565 of 5985
on 566 of 5985
on 567 of 5985
on 568 of 5985
on 569 of 5985
on 570 of 5985
on 571 of 5985
on 572 of 5985
on 573 of 5985
on 574 of 5985
on 575 of 5985
on 576 of 5985
on 577 of 5985
on 578 of 5985
on 579 of 5985
on 580 of 5985
on 581 of 5985
on 582 of 5985
on 583 of 5985
on 584 of 5985
on 585 of 5985
on 586 of 5985
on 587 of 5985
on 588 of 5985
on 589 of 5985
on 590 of 5985
on 591 of 5985
on 592 of 5985
on 593 of 5985
on 594 of 5985
on 595 of 5985
on 596 of 5985
on 597 of 5985
on 598 of 5985
on 599 of 5985
on 600 of 5985
on 601 of 5985
on 602 of 5985
on 603 of 5985
on 604 of 5985
on 605 of 5985
on 606 of 5985
on 607 of 5985
on 608 of 5985
on 609 of 5985
on 610 of 5985
on 611 of 5985
on 612 of 5985
on 613 of 5985
on 614 of 5985
on 615 of 5985
on 616 of 5985
on 617 of 5985
on 618 of 5985
on 619 of 5985
on 620 of 5985
on 621 of 5985
on 622 of 

on 1096 of 5985
on 1097 of 5985
on 1098 of 5985
on 1099 of 5985
on 1100 of 5985
on 1101 of 5985
on 1102 of 5985
on 1103 of 5985
on 1104 of 5985
on 1105 of 5985
on 1106 of 5985
on 1107 of 5985
on 1108 of 5985
on 1109 of 5985
on 1110 of 5985
on 1111 of 5985
on 1112 of 5985
on 1113 of 5985
on 1114 of 5985
on 1115 of 5985
on 1116 of 5985
on 1117 of 5985
on 1118 of 5985
on 1119 of 5985
on 1120 of 5985
on 1121 of 5985
on 1122 of 5985
on 1123 of 5985
on 1124 of 5985
on 1125 of 5985
on 1126 of 5985
on 1127 of 5985
on 1128 of 5985
on 1129 of 5985
on 1130 of 5985
on 1131 of 5985
on 1132 of 5985
on 1133 of 5985
on 1134 of 5985
on 1135 of 5985
on 1136 of 5985
on 1137 of 5985
on 1138 of 5985
on 1139 of 5985
on 1140 of 5985
on 1141 of 5985
on 1142 of 5985
on 1143 of 5985
on 1144 of 5985
on 1145 of 5985
on 1146 of 5985
on 1147 of 5985
on 1148 of 5985
on 1149 of 5985
on 1150 of 5985
on 1151 of 5985
on 1152 of 5985
on 1153 of 5985
on 1154 of 5985
on 1155 of 5985
on 1156 of 5985
on 1157 of 5985
on 1158 

...

on 3661 of 5985
on 3662 of 5985
on 3663 of 5985
on 3664 of 5985
on 3665 of 5985
on 3666 of 5985
on 3667 of 5985
on 3668 of 5985
on 3669 of 5985
on 3670 of 5985
on 3671 of 5985
on 3672 of 5985
on 3673 of 5985
on 3674 of 5985
on 3675 of 5985
on 3676 of 5985
on 3677 of 5985
on 3678 of 5985
on 3679 of 5985
on 3680 of 5985
on 3681 of 5985
on 3682 of 5985
on 3683 of 5985
on 3684 of 5985
on 3685 of 5985
on 3686 of 5985
on 3687 of 5985
on 3688 of 5985
on 3689 of 5985
on 3690 of 5985
on 3691 of 5985
on 3692 of 5985
on 3693 of 5985
on 3694 of 5985
on 3695 of 5985
on 3696 of 5985
on 3697 of 5985
on 3698 of 5985
on 3699 of 5985
on 3700 of 5985
on 3701 of 5985
on 3702 of 5985
on 3703 of 5985
on 3704 of 5985
on 3705 of 5985
on 3706 of 5985
on 3707 of 5985
on 3708 of 5985
on 3709 of 5985
on 3710 of 5985
on 3711 of 5985
on 3712 of 5985
on 3713 of 5985
on 3714 of 5985
on 3715 of 5985
on 3716 of 5985
on 3717 of 5985
on 3718 of 5985
on 3719 of 5985
on 3720 of 5985
on 3721 of 5985
on 3722 of 5985
on 3723 

on 4174 of 5985
on 4175 of 5985
on 4176 of 5985
on 4177 of 5985
on 4178 of 5985
on 4179 of 5985
on 4180 of 5985
on 4181 of 5985
on 4182 of 5985
on 4183 of 5985
on 4184 of 5985
on 4185 of 5985
on 4186 of 5985
on 4187 of 5985
on 4188 of 5985
on 4189 of 5985
on 4190 of 5985
on 4191 of 5985
on 4192 of 5985
on 4193 of 5985
on 4194 of 5985
on 4195 of 5985
on 4196 of 5985
on 4197 of 5985
on 4198 of 5985
on 4199 of 5985
on 4200 of 5985
on 4201 of 5985
on 4202 of 5985
on 4203 of 5985
on 4204 of 5985
on 4205 of 5985
on 4206 of 5985
on 4207 of 5985
on 4208 of 5985
on 4209 of 5985
on 4210 of 5985
on 4211 of 5985
on 4212 of 5985
on 4213 of 5985
on 4214 of 5985
on 4215 of 5985
on 4216 of 5985
on 4217 of 5985
on 4218 of 5985
on 4219 of 5985
on 4220 of 5985
on 4221 of 5985
on 4222 of 5985
on 4223 of 5985
on 4224 of 5985
on 4225 of 5985
on 4226 of 5985
on 4227 of 5985
on 4228 of 5985
on 4229 of 5985
on 4230 of 5985
on 4231 of 5985
on 4232 of 5985
on 4233 of 5985
on 4234 of 5985
on 4235 of 5985
on 4236 

on 4687 of 5985
on 4688 of 5985
on 4689 of 5985
on 4690 of 5985
on 4691 of 5985
on 4692 of 5985
on 4693 of 5985
on 4694 of 5985
on 4695 of 5985
on 4696 of 5985
on 4697 of 5985
on 4698 of 5985
on 4699 of 5985
on 4700 of 5985
on 4701 of 5985
on 4702 of 5985
on 4703 of 5985
on 4704 of 5985
on 4705 of 5985
on 4706 of 5985
on 4707 of 5985
on 4708 of 5985
on 4709 of 5985
on 4710 of 5985
on 4711 of 5985
on 4712 of 5985
on 4713 of 5985
on 4714 of 5985
on 4715 of 5985
on 4716 of 5985
on 4717 of 5985
on 4718 of 5985
on 4719 of 5985
on 4720 of 5985
on 4721 of 5985
on 4722 of 5985
on 4723 of 5985
on 4724 of 5985
on 4725 of 5985
on 4726 of 5985
on 4727 of 5985
on 4728 of 5985
on 4729 of 5985
on 4730 of 5985
on 4731 of 5985
on 4732 of 5985
on 4733 of 5985
on 4734 of 5985
on 4735 of 5985
on 4736 of 5985
on 4737 of 5985
on 4738 of 5985
on 4739 of 5985
on 4740 of 5985
on 4741 of 5985
on 4742 of 5985
on 4743 of 5985
on 4744 of 5985
on 4745 of 5985
on 4746 of 5985
on 4747 of 5985
on 4748 of 5985
on 4749 

on 5200 of 5985
on 5201 of 5985
on 5202 of 5985
on 5203 of 5985
on 5204 of 5985
on 5205 of 5985
on 5206 of 5985
on 5207 of 5985
on 5208 of 5985
on 5209 of 5985
on 5210 of 5985
on 5211 of 5985
on 5212 of 5985
on 5213 of 5985
on 5214 of 5985
on 5215 of 5985
on 5216 of 5985
on 5217 of 5985
on 5218 of 5985
on 5219 of 5985
on 5220 of 5985
on 5221 of 5985
on 5222 of 5985
on 5223 of 5985
on 5224 of 5985
on 5225 of 5985
on 5226 of 5985
on 5227 of 5985
on 5228 of 5985
on 5229 of 5985
on 5230 of 5985
on 5231 of 5985
on 5232 of 5985
on 5233 of 5985
on 5234 of 5985
on 5235 of 5985
on 5236 of 5985
on 5237 of 5985
on 5238 of 5985
on 5239 of 5985
on 5240 of 5985
on 5241 of 5985
on 5242 of 5985
on 5243 of 5985
on 5244 of 5985
on 5245 of 5985
on 5246 of 5985
on 5247 of 5985
on 5248 of 5985
on 5249 of 5985
on 5250 of 5985
on 5251 of 5985
on 5252 of 5985
on 5253 of 5985
on 5254 of 5985
on 5255 of 5985
on 5256 of 5985
on 5257 of 5985
on 5258 of 5985
on 5259 of 5985
on 5260 of 5985
on 5261 of 5985
on 5262 

on 5713 of 5985
on 5714 of 5985
on 5715 of 5985
on 5716 of 5985
on 5717 of 5985
on 5718 of 5985
on 5719 of 5985
on 5720 of 5985
on 5721 of 5985
on 5722 of 5985
on 5723 of 5985
on 5724 of 5985
on 5725 of 5985
on 5726 of 5985
on 5727 of 5985
on 5728 of 5985
on 5729 of 5985
on 5730 of 5985
on 5731 of 5985
on 5732 of 5985
on 5733 of 5985
on 5734 of 5985
on 5735 of 5985
on 5736 of 5985
on 5737 of 5985
on 5738 of 5985
on 5739 of 5985
on 5740 of 5985
on 5741 of 5985
on 5742 of 5985
on 5743 of 5985
on 5744 of 5985
on 5745 of 5985
on 5746 of 5985
on 5747 of 5985
on 5748 of 5985
on 5749 of 5985
on 5750 of 5985
on 5751 of 5985
on 5752 of 5985
on 5753 of 5985
on 5754 of 5985
on 5755 of 5985
on 5756 of 5985
on 5757 of 5985
on 5758 of 5985
on 5759 of 5985
on 5760 of 5985
on 5761 of 5985
on 5762 of 5985
on 5763 of 5985
on 5764 of 5985
on 5765 of 5985
on 5766 of 5985
on 5767 of 5985
on 5768 of 5985
on 5769 of 5985
on 5770 of 5985
on 5771 of 5985
on 5772 of 5985
on 5773 of 5985
on 5774 of 5985
on 5775 